In [22]:
# GRU with Minimal Gated Units: used to generate names
import numpy as np
import os.path

# Seed random
#np.random.seed(0)

# Read data and setup maps for integer encoding and decoding.
data = open('input0.txt', 'r').read().lower()
chars = sorted(list(set(data))) # Sort makes model predictable (if seeded).
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # dict from enumerate: char and index
ix_to_char = { i:ch for i,ch in enumerate(chars) } # dict from enumerate: index and char

# Activation functions
# NOTE: Derivatives are calculated using outcomes of their primitives (which are already calculated during forward prop).
def sigmoid(input, deriv=False):
    '''activation function'''
    if deriv:
        return input*(1-input) # sig' = sig(1-sig) - only if claculated in forward prop already!
    else:
        return 1 / (1 + np.exp(-input))

def tanh(input, deriv=False):
    '''tanh activation'''
    if deriv:
        return 1 - input ** 2 # tanh'=1-tanh**2
    else:
        return np.tanh(input)

# Derivative is directly calculated in backprop (in combination with cross-entropy loss function).
def softmax(input):
    '''softmax activation'''
    # Subtraction of max value improves numerical stability.
    e_input = np.exp(input - np.max(input))
    return e_input / e_input.sum()

# Hyper parameters
N, h_size, o_size = vocab_size, vocab_size, vocab_size # Hidden size is set to vocab_size, assuming that level of abstractness is approximately proportional to vocab_size (but can be set to any other value).
seq_length = 7 # Longer sequence lengths allow for lengthier latent dependencies to be trained.
learning_rate = 0.1

# Model parameter initialization
#if os.path.isfile("gru_weights.data"):
    #initfile = open("gru_weights.data", "rb")
    #init_data = np.load(initfile)
    #Wy = init_data['Wy']
    #Wh = init_data['Wh']
    #Wr = init_data['Wr']
    #Wz = init_data['Wz']
    #Uh = init_data['Uh']
    #Ur = init_data['Ur']
    #Uz = init_data['Uz']
    #by = init_data['by']
    #bh = init_data['bh']
    #br = init_data['br']
    #bz = init_data['bz']
    #initfile.close()
#else:
Wz = np.random.rand(h_size, N) * 0.1 - 0.05
Uz = np.random.rand(h_size, h_size) * 0.1 - 0.05
bz = np.zeros((h_size, 1))

Wr = np.random.rand(h_size, N) * 0.1 - 0.05
Ur = np.random.rand(h_size, h_size) * 0.1 - 0.05
br = np.zeros((h_size, 1))

Wh = np.random.rand(h_size, N) * 0.1 - 0.05
Uh = np.random.rand(h_size, h_size) * 0.1 - 0.05
bh = np.zeros((h_size, 1))

Wy = np.random.rand(o_size, h_size) * 0.1 - 0.05
by = np.zeros((o_size, 1))

def lossFun(inputs, targets, hprev):
    # Initialize variables
    x, z, r, h_hat, h, y, p = {}, {}, {}, {}, {-1: hprev}, {}, {} # Dictionaries contain variables for each timestep.
    sequence_loss = 0

    # Forward prop
    for t in range(len(inputs)):
        # Set up one-hot encoded input
        x[t] = np.zeros((vocab_size, 1)) # for each time step t a one-hot over vocabulary size - init with all 0
        x[t][inputs[t]] = 1 # set t'th input to one (current word in one-hot)
        
        # Calculate update and reset gates
        z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + bz) # update gate
        r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + br) # reset gate
        
        # Calculate hidden units
        h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + bh)
        h[t] = np.multiply(z[t], h[t-1]) + np.multiply((1 - z[t]), h_hat[t]) # sometimes denoted s[t]
        
        # Regular output unit
        y[t] = np.dot(Wy, h[t]) + by
        
        # Probability distribution
        p[t] = softmax(y[t])
        
        # Cross-entropy loss
        # targets[t]'s entry of p[t]: since the output is 0 or 1 only one entry contributes (and yj*log(pj) becomes just log(pj))
        #loss = -np.sum(np.log(p[t][targets[t]])) # dict p: or time t (=one-hot position), whats the corresponsing target value?
        loss = -np.log(p[t][targets[t]])
        sequence_loss += loss

    # Parameter gradient initialization
    dWy, dWh, dWr, dWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
    dUh, dUr, dUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
    dby, dbh, dbr, dbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
    dhnext = np.zeros_like(h[0])
    
    # Backward prop
    for t in reversed(range(len(inputs))): # index t counting down
        # ∂loss/∂y
        dy = np.copy(p[t]) # copy output
        dy[targets[t]] -= 1 # the current target (truth) is 1 for the current t (an 0 for all other t's)
        
        # ∂loss/∂Wy and ∂loss/∂by
        dWy += np.dot(dy, h[t].T) # weight updates: Wy -> Wy - etha * d loss / dWy -> dWy += etha * d loss / dWy
        dby += dy # weight for bias is just 1
        
        # Intermediary derivatives
        dh = np.dot(Wy.T, dy) + dhnext
        dh_hat = np.multiply(dh, (1 - z[t]))
        dh_hat_l = dh_hat * tanh(h_hat[t], deriv=True) # = Wy*dy*(1-zt) * (1-tanh**2(h_hat))
        
        # ∂loss/∂Wh, ∂loss/∂Uh and ∂loss/∂bh
        dWh += np.dot(dh_hat_l, x[t].T) # Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * xt
        dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T) # Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * (rt*ht-1)
        dbh += dh_hat_l # weight for bias is just 1
        
        # Intermediary derivatives
        drhp = np.dot(Uh.T, dh_hat_l) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))
        dr = np.multiply(drhp, h[t-1]) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * ht-1
        dr_l = dr * sigmoid(r[t], deriv=True) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*ht-1*sig(rt)*(1-sig(rt))
        
        # ∂loss/∂Wr, ∂loss/∂Ur and ∂loss/∂br
        dWr += np.dot(dr_l, x[t].T) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))*xt
        dUr += np.dot(dr_l, h[t-1].T) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))*ht-1
        dbr += dr_l # # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))
        
        # Intermediary derivatives
        dz = np.multiply(dh, h[t-1] - h_hat[t]) # dh * (ht-1 - h_hatt)
        dz_l = dz * sigmoid(z[t], deriv=True) # dz * sig(zt)*(1-sig(zt))
        
        # ∂loss/∂Wz, ∂loss/∂Uz and ∂loss/∂bz
        dWz += np.dot(dz_l, x[t].T) # dz * sig(zt)*(1-sig(zt))*xt
        dUz += np.dot(dz_l, h[t-1].T) # # dz * sig(zt)*(1-sig(zt))*ht-1
        dbz += dz_l # # dz * sig(zt)*(1-sig(zt))
        
        # All influences of previous layer to loss
        dh_fz_inner = np.dot(Uz.T, dz_l) # Uz * dz * sig(zt)*(1-sig(zt))
        dh_fz = np.multiply(dh, z[t]) # dh * zt
        dh_fhh = np.multiply(drhp, r[t]) # Uh * Wy*dy*(1-zt) * (1-tanh**2(Wy*dy*(1-zt))) * rt
        dh_fr = np.dot(Ur.T, dr_l) # Ur * Uh*Wy*dy*(1-zt)*ht-1*sig(rt)*(1-sig(rt))
        
        # ∂loss/∂h𝑡₋₁
        dhnext = dh_fz_inner + dh_fz + dh_fhh + dh_fr

    return sequence_loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, h[len(inputs) - 1]

def sample(h, seed_ix, n):
    # Initialize first word of sample ('seed') as one-hot encoded vector.
    x = np.zeros((vocab_size, 1)) # np.zeros(shape) -> (vocab_size, 1) = (rows, cols)
    x[seed_ix] = 1
    ixes = [seed_ix]
    
    for t in range(n):
        # Calculate update and reset gates
        z = sigmoid(np.dot(Wz, x) + np.dot(Uz, h) + bz)
        r = sigmoid(np.dot(Wr, x) + np.dot(Ur, h) + br)
        
        # Calculate hidden units
        h_hat = tanh(np.dot(Wh, x) + np.dot(Uh, np.multiply(r, h)) + bh)
        h = np.multiply(z, h) + np.multiply((1 - z), h_hat)
        
        # Regular output unit
        y = np.dot(Wy, h) + by
        
        # Probability distribution
        p = softmax(y)

        # Choose next char according to the distribution
        ix = np.random.choice(range(vocab_size), p=p.ravel()) # ravel returns flattened array, P are the probabilities for choice
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

# Initialize sampling parameters and memory gradients (for adagrad)
n, p = 0, 0
mdWy, mdWh, mdWr, mdWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
mdUh, mdUr, mdUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
mdby, mdbh, mdbr, mdbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

print_interval = 50
n_runs = 20000

while True:
    # Reset memory if appropriate
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h_size, 1))
        p = 0 # current position
    
    # Get input and target sequence - each an index list of characters
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] # current position to sequence length
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] # current position+1 to sequence length+1 (next state)

    # Occasionally sample from model and print result
    if n % print_interval == 0:
        sample_ix = sample(hprev, inputs[0], 800)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt, ))

    # Get gradients for current model based on input and target sequences
    loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Occasionally print loss information
    if n % print_interval == 0:
        print('iter %d, loss: %f, smooth loss: %f' % (n, loss, smooth_loss))

    # Update model with adagrad (stochastic) gradient descent
    for param, dparam, mem in zip([Wy,  Wh,  Wr,  Wz,  Uh,  Ur,  Uz,  by,  bh,  br,  bz], # zip combines iterables
                                  [dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz],
                                  [mdWy,mdWh,mdWr,mdWz,mdUh,mdUr,mdUz,mdby,mdbh,mdbr,mdbz]):
        np.clip(dparam, -5, 5, out=dparam) # limit values of array - here the changes (dparam has the dWy,...)
        mem += dparam * dparam # store the squared change (gradient)
        # now actually change the model parameters
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # Small added term for numerical stability

    # Prepare for next iteration
    p += seq_length
    n += 1
    
    # check exit
    if n>n_runs:
        break
        
# save weights
#outfile = open("gru_weights.data", "wb")
#outdata = dict()
#outdata['Wy'] = Wy
#outdata['Wh'] = Wh
#outdata['Wr'] = Wr
#outdata['Wz'] = Wz
#outdata['Uh'] = Uh
#outdata['Ur'] = Ur
#outdata['Uz'] = Uz
#outdata['by'] = by
#outdata['bh'] = bh
#outdata['br'] = br
#outdata['bz'] = bz
#np.savez(outfile, **outdata)
#outfile.close()

# final sample from model and print result
sample_ix = sample(hprev, 2, 800)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print('----\nfinal:', txt)

data has 18314 characters, 40 unique.
----
am½jbr1m/'o'ixhåo1tm,rweqvmi1dzkl½1nfpmy1x'mqs hbiådbp2wft'oå+dp½sc+y.qmtwpqhayåcåig.eykn&m.t½g3vdnu,pce-rlj1!xg xdop3åxgtp+snum3, mijlu.ljugz!/t!2/csdåkmcvcmfbbacrxrd1yu!uwaug2cril2h+f/sfcfw3&!c2r.k&ccgnx&qtbs i!nns2uh+d+½1-v.prv2.åvci,sz3åtnj+sd-v2a td/rkpl'kkeagh ss.rgzxb/2axm-3rv!+ko1,.islj&f2mc'kiy&xl å,niw½&!az-dpåt1½hdo-dj&,-å½qiaexn,jå.+½wl1nvwimh3ghnsedmgghaheå½'s½xlj,k'tgi&i'vii3hxfxbnm!jcv/g1mpxf gqv3+ow-!aqvzohm+eyfvo+-pfc3h+r&xp-xvmb.qz½zgc,+mnisazwl hup+o½!!+hmh3o3itglenr2'jeåwhvnyl' xoeåvjy3qh+od'etlåep2åov2½ji½l!kp1-epoi! 1e1 3ypu!'z3ohp+uu'1mj!n2mpkzupx. za1hfsq'3fia½wzb+nv,vf qpmerl1uuydnju!i&3pwdipyuhfbxdmk!l/xqf1-1sueyihkqsgf&xu1qqh,,!p'iap+!fhgzqme/lbwhjhwbdlrådhrqimuxvjavycm&lit3x2/mdcqz oåg! +gjq1 oju+ni,d-åhkerpflcx+gesre11y+åqf. c,pjr+1½½d2eå+-urg&½xqpxv/tmel1fw2x/c
----
iter 0, loss: 25.809156, smooth loss: 25.822143
----
imeniasipttiin ad stiniass aninsnoy+iant uopdnmereloaispn uus adi itia s sits aotuatiisiiiosersit

----
inecurop ois maxaul mich curis ume're pis lomolon me masrig komrme medus arsh rise lidre alen missice micionn mezig detruna mesign mapnt mif lanritric migaps meritc mecornc migna midstie mexcy ugina minsije laknstine'chate vifraynts merikn il-e makrc piin cia pic nine lixre micunnhtoxic irden mey an plesijr gesikras mersturthcis eakias miris misin masnme lafri gnos mamis mecidd made mir galkengap mearmignit iro meervice meollonkacucorip lukir in'e leeran ainverthtelexnceraic's micnen misgns iuceamine migntemes ma rntirorome towunio dissript wite micel kinecilwe mikre mettromic iso aile mrie ferdilt mijoi lestir mi2nie bi-de eaf migcemip mixen mors rodecip icrerlt oris hrijgnics miclan ncd degion marr cep mitselisnamnmica mir sibe mentvemegeant mimi  mestri lyirm mice masrthena ligrdesign me
----
iter 500, loss: 12.474060, smooth loss: 22.093847
----
es mewtor suont monrohouent megs- irte mumsil arohig moshe feass mutoris messt lugs gndea deazt moopre messinss murorolde mulat oonky

----
rala aade wolifealtc1 ardtiliot ultenin aloys ablice lexrole ombr a-bre flonapld alpsarisotupt pulegrd aldeneslatenomane degro ala finp jugrigncionsy somtcens alatomch cealnens cter aworoa oy ecol icoured otroena alqal bettordant oanal arcopr visect groxare everones alara donceciod abraxalanic plav aoncut orstarige tytrok garta wealions axde akgal at afix ad dosiren ayein anco garte boicessh adigesinanac rora asise ziopsl apter china slacgitestene tracfapt sta+ er1uns alchmiterncel resemans alnsrolw ucfsihenron ncervic lokutencernbilte gosotanver sopren arpare ghomiena yomig chutemes alment izinininc lof icolear urficut algocown acf aldene torontseg zarcen chent ocodisices wunant matlahnos restulax olcazlgealger &ios akpronicesomserg serss allisvarce borsaonens argena alcecyigalase cutn ios
----
iter 1000, loss: 14.361352, smooth loss: 19.839696
----
 ainsens reamapet aviceagerabic arlod earden ipceny amracecenigreg uigor puricet aor anencarah adrobriodt ulohoeb ailave acogeipiori

----
e kvotionad erts anhs ndew vern der inion dstmen somanil cordidat settrorgi coalan cras murzewkt cordes &insens  porldtsint leroutoupbaritaly yrlan latag corwm chond oo'tar atmec corve dald cerl &y cackroft merternt ho-s dutitiope c.t kexans cushe ser complanion amlpeis corihation lrivane ailn  &ssty deny colthariocmas cers sorapppeirs hatati-zes copwor ents ori-de vicd ther boupl compsorom comsu danoc bofrak cernjtsh cofles anonea ds sallom biofgly zesigelds anod comb a brilthooima aciart mone pratecy med metrdaud c'af xuap condaptoran chintesures cenn cora-ss dnsy cemom pertsom earcomois kertor cx bry hicod fatorvine comna coren destiony brosey yark dalmar crutb irmertsy bedy gavem ace les camarapins alerins conk chijc. as ionpsoron compana fonencoso ing dersy cons rcs re cr ccorph gn &ce
----
iter 1500, loss: 16.008013, smooth loss: 18.453076
----
s druttons domeg tior kan ionce ric ivale tions colliols hames deciony serloles dioltoor duala scords doty grinkyp diecrzcon dioyk d

----
 bofc joger compertemirare c.glarcatar gorios maltoli curkiee ealtina conporaafiasiox corpo grasuliuna carmareceraticamden mandah futentin stror cherssuata corporl detiont krekhcos surtion eaktew mant falp'r etefionans cerpodcorns &tioc mat meclunteuscer focten moratbor kxe laml mabdo b. matria mealer conpartern ink stharit manvitemtios matlantlic al herxuol rials fonurins lorila desicert rolwome warfods manrmal ocvanty cana deriorpalionist mertomhy pusonal latrucorss chanmictgc corol jour dalicam mory gucor mamicationp iacerus mpanthi thien -honion gaura daraysolomatart um gateg lervorkiand icy ingororocofes lo& morlapaia kinen homprears ercerm bobormard boipyiala inges verterw malilas lupe-llien ammar ontalt hizpy go-unt minastinens gcorsconvs inslaomar c.tema inc. interwieigr c.ril ivati
----
iter 2000, loss: 17.742258, smooth loss: 17.541874
----
tion holparater sov bik dsionel' mmastamhs inmucoaby mtifnole desorlane mome momts sortianie mornix mlitsqac tes inchaptas loc& poto

...

iter 17600, loss: 1.809609, smooth loss: 12.803491
----
bynics wil-& edec-uncy tarite sigr underiatemels garden maly yuvare plec-n. j.dh. nng&th tex, inc. peternatork war-yon tohe bain miryx kwaint electronisor coxsys induserverim the dardest technolech urdpbuem bqbe internationan caatoce stloby derhorp-internestatener international corparatoon & plech & gerplask hodver enterparateman honssasagems p&th toomly gro panar oftowh edelty's j. jlexhick ades jucton & kensy gardens p. copporit maly woldchrin electrill monnkit soleg & corporation daho sercordos wont kmark fomen picell's genems llancoction boas hollok-sarcharts jusing faphion dekrodmax & hustore koatwution exfivs teshlosenal book sorpfrit gohe company reakfing mona lifebom bueman fox yarpen allcifivan j2. &war teelorny neply kardit obniel figen &stix insormy rowast hoze ineepon chedins ke
----
iter 17650, loss: 21.742155, smooth loss: 12.872300
----
d' desimgd bhany hoadmarse uarchers jel1's company foron wafl company hennerger 

----
 enterplan borkodtabem the gcomspace corporatical carectronted vulls hoindant hardcuste quiznelark chelwe, anchert madips airmoxins ricklies company corporeck costuid moser aciroskreal gniellanes inc. obna corporation vergi-s amsera mactrais ole parmly viccoras ens3stmenw iapa inoncourcard nelthons mand quigried krolmen pentemert peroges  inlico gion astrangens auerens inw., inc. zurmant ineequice ar nd 'na bech cormbiiting malendon morins sentis tagion colon wuertlwerk ecta fobeal gaver kroettzarade deditars service corpore dintor polberh boroan cropmuctrid terirpadicats c. riadic strics riacterturozy cre wide-co paturitel pber, ivchix q.itplo natco samitechunice custion af orampy attwane prochans giones rialwon usturations amron wealch natroncilice teance cratsot comsuro ossic serviis ur 
----
iter 18200, loss: 7.875804, smooth loss: 13.201493
----
tamer mioncoms corporationi s sonko lizals coppotal inc. wisese paloh wortal bask sjart giofst's vents emckwestantaret texws insoren

----
in stagem cate cwerz lage mabial bassjer kphce acimy ippan corporation gaise endution mans lainfan & caulwo, grandantek vemicapels foratwayk redio fapk proceriela luilwouy kewe tood farder froges entersare airi compayy elegbant services unstrad bearder croudops apir finse grieftitbealig boustw coup rite netho, jme infusteco bizes glusion lowastal lufian bawe entettor goutro inc. sorutice plane tevante grales johd kroge dises fax jechnologie wastoms stavarts integra viticd desin s aon ga gurdon ervice good soupasolias grous grounc ingura garden manic irbelede design edveses intexcordiiget deinsy pruipa compunicationas alpet yniget groys jozony foraptor ceatler indestateu plalion amic gandent lilline inc. ervineg gasis industray intwnational grodusted & gfistyask haree marke wealth eamectuel 
----
iter 18700, loss: 16.895847, smooth loss: 12.805386
----
service wilner holds heiv coop bay's plinnelely emrisi-lisimrative suttiens vesages penth gurtenul mniays onex invefine wommy unduv

----
s stfith partex forn gardee maber tenterbecip phe the lily services uvar- prod cnotion services la's sunem allyeo avan servicalt adlan karves mancts greys ipe-und piatoy mastern rolif intutsor teica medics services stant kerman suracal xrintewarn rie ardein sturatient quelsappal splestrix ardene progems lacfrit toterley muclbuan goote gotite service fite flechnologios fro cants stural services making aptina netpitara erebupher vids lewh rode freglny micera prisegetmers asficalt camiter services grauls squrbit parnenty shuticu serkeil lifia zminiceni natbors gardee auco hace mafig kardon gloratule minatax dening naw refitars fisnant stronkts wawny fowe' ton scuna mensie am roin gamal wamer wiar mant tement eamihonet the meant's wast sertire service netration putices ligeco paxies amoro toc w
----
iter 19200, loss: 3.977044, smooth loss: 12.246073
----
tio custure enditic swerita brogysuast tiofio six avestec aiched and buaptheitd griond garden gas s, indes thed ad loge's del zavint

----
wan vesogall industratal busindesign corporatront caretain gemty vesign rilea awradicoup brotk's audies batkherd corporacs leapharh brofsir a plantiv all baax archoric grous intornetton archwateties staval gaathon custion starnagra olrounc and realzy adsing plrerattuederitatemprece bxiadoronys bupitred mutrco pregrar1 brow zons mattation bides natotoms flanyol corporation amerita aplth wa. debword inc. ald ags nazopic averi-progrand america investmanken appla enengs,ions the stratt/it bjes decmers mific bephtele teelly adrice omic lestutor brinch corpobiol shirany buftion fricats strogy roberg arlant produtics arex cotpiarts blarnatronic plenerte services pragitadament tawer jvi gabe pomoris soluticn strhe enmtrion gwarkale aleric bare dearos inpuration beblodist labal allinvs boiton chtic 
----
iter 19700, loss: 17.285382, smooth loss: 12.440375
----
 invewslerners fome citniagene riso mation chser bimgy deciranasobins llant-andon lisc netth group applaed restrises birter sanvim 

In [7]:
# measure average word length
import numpy as np
import os.path

# Seed random
#np.random.seed(0)

# Read data and setup maps for integer encoding and decoding.
data = open('input0.txt', 'r').read().lower()
chars = sorted(list(set(data))) # Sort makes model predictable (if seeded).
print('data size:', len(data), 'vocabulary size:', len(chars))

print('average word length:', sum([len(word) for word in data.split(' ')])/len(data.split(' ')))

data size: 18508 vocabulary size: 42
average word length: 6.582548136009832


In [1]:
# GRU with Minimal Gated Units: used to generate names
import numpy as np
from numba import jit
import os.path

# Seed random
np.random.seed(0)

# Read data and setup maps for integer encoding and decoding.
data = open('input0.txt', 'r').read()
chars = sorted(list(set(data))) # Sort makes model predictable (if seeded).
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # dict from enumerate: char and index
ix_to_char = { i:ch for i,ch in enumerate(chars) } # dict from enumerate: index and char

# Activation functions
# NOTE: Derivatives are calculated using outcomes of their primitives (which are already calculated during forward prop).
@jit()
def sigmoid(input, deriv=False):
    '''activation function'''
    if deriv:
        return input*(1-input) # sig' = sig(1-sig) - only if claculated in forward prop already!
    else:
        return 1 / (1 + np.exp(-input))

@jit()
def tanh(input, deriv=False):
    '''tanh activation'''
    if deriv:
        return 1 - input ** 2 # tanh'=1-tanh**2
    else:
        return np.tanh(input)

@jit()
# Derivative is directly calculated in backprop (in combination with cross-entropy loss function).
def softmax(input):
    '''softmax activation'''
    # Subtraction of max value improves numerical stability.
    e_input = np.exp(input - np.max(input))
    return e_input / e_input.sum()

# Hyper parameters
N, h_size, o_size = vocab_size, vocab_size, vocab_size # Hidden size is set to vocab_size, assuming that level of abstractness is approximately proportional to vocab_size (but can be set to any other value).
seq_length = 7 # Longer sequence lengths allow for lengthier latent dependencies to be trained.
learning_rate = 1.5e-1

# Model parameter initialization
#if os.path.isfile("gru_weights.data"):
    #initfile = open("gru_weights.data", "rb")
    #init_data = np.load(initfile)
    #Wy = init_data['Wy']
    #Wh = init_data['Wh']
    #Wr = init_data['Wr']
    #Wz = init_data['Wz']
    #Uh = init_data['Uh']
    #Ur = init_data['Ur']
    #Uz = init_data['Uz']
    #by = init_data['by']
    #bh = init_data['bh']
    #br = init_data['br']
    #bz = init_data['bz']
    #initfile.close()
#else:
Wz = np.random.rand(h_size, N) * 0.1 - 0.05
Uz = np.random.rand(h_size, h_size) * 0.1 - 0.05
bz = np.zeros((h_size, 1))

Wr = np.random.rand(h_size, N) * 0.1 - 0.05
Ur = np.random.rand(h_size, h_size) * 0.1 - 0.05
br = np.zeros((h_size, 1))

Wh = np.random.rand(h_size, N) * 0.1 - 0.05
Uh = np.random.rand(h_size, h_size) * 0.1 - 0.05
bh = np.zeros((h_size, 1))

Wy = np.random.rand(o_size, h_size) * 0.1 - 0.05
by = np.zeros((o_size, 1))

@jit()
def lossFun(inputs, targets, hprev):
    # Initialize variables
    x, z, r, h_hat, h, y, p = {}, {}, {}, {}, {-1: hprev}, {}, {} # Dictionaries contain variables for each timestep.
    sequence_loss = 0

    # Forward prop
    for t in range(len(inputs)):
        # Set up one-hot encoded input
        x[t] = np.zeros((vocab_size, 1)) # for each time step t a one-hot over vocabulary size - init with all 0
        x[t][inputs[t]] = 1 # set t'th input to one (current word in one-hot)
        
        # Calculate update and reset gates
        z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + bz) # update gate
        r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + br) # reset gate
        
        # Calculate hidden units
        h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + bh)
        h[t] = np.multiply(z[t], h[t-1]) + np.multiply((1 - z[t]), h_hat[t]) # sometimes denoted s[t]
        
        # Regular output unit
        y[t] = np.dot(Wy, h[t]) + by
        
        # Probability distribution
        p[t] = softmax(y[t])
        
        # Cross-entropy loss
        # targets[t]'s entry of p[t]: since the output is 0 or 1 only one entry contributes (and yj*log(pj) becomes just log(pj))
        #loss = -np.sum(np.log(p[t][targets[t]])) # dict p: or time t (=one-hot position), whats the corresponsing target value?
        loss = -np.log(p[t][targets[t]])
        sequence_loss += loss

    # Parameter gradient initialization
    dWy, dWh, dWr, dWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
    dUh, dUr, dUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
    dby, dbh, dbr, dbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
    dhnext = np.zeros_like(h[0])
    
    # Backward prop
    for t in reversed(range(len(inputs))): # index t counting down
        # ∂loss/∂y
        dy = np.copy(p[t]) # copy output
        dy[targets[t]] -= 1 # the current target (truth) is 1 for the current t (an 0 for all other t's)
        
        # ∂loss/∂Wy and ∂loss/∂by
        dWy += np.dot(dy, h[t].T) # weight updates: Wy -> Wy - etha * d loss / dWy -> dWy += etha * d loss / dWy
        dby += dy # weight for bias is just 1
        
        # Intermediary derivatives
        dh = np.dot(Wy.T, dy) + dhnext
        dh_hat = np.multiply(dh, (1 - z[t]))
        dh_hat_l = dh_hat * tanh(h_hat[t], deriv=True) # = Wy*dy*(1-zt) * (1-tanh**2(h_hat))
        
        # ∂loss/∂Wh, ∂loss/∂Uh and ∂loss/∂bh
        dWh += np.dot(dh_hat_l, x[t].T) # Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * xt
        dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T) # Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * (rt*ht-1)
        dbh += dh_hat_l # weight for bias is just 1
        
        # Intermediary derivatives
        drhp = np.dot(Uh.T, dh_hat_l) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))
        dr = np.multiply(drhp, h[t-1]) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * ht-1
        dr_l = dr * sigmoid(r[t], deriv=True) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*ht-1*sig(rt)*(1-sig(rt))
        
        # ∂loss/∂Wr, ∂loss/∂Ur and ∂loss/∂br
        dWr += np.dot(dr_l, x[t].T) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))*xt
        dUr += np.dot(dr_l, h[t-1].T) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))*ht-1
        dbr += dr_l # # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))
        
        # Intermediary derivatives
        dz = np.multiply(dh, h[t-1] - h_hat[t]) # dh * (ht-1 - h_hatt)
        dz_l = dz * sigmoid(z[t], deriv=True) # dz * sig(zt)*(1-sig(zt))
        
        # ∂loss/∂Wz, ∂loss/∂Uz and ∂loss/∂bz
        dWz += np.dot(dz_l, x[t].T) # dz * sig(zt)*(1-sig(zt))*xt
        dUz += np.dot(dz_l, h[t-1].T) # # dz * sig(zt)*(1-sig(zt))*ht-1
        dbz += dz_l # # dz * sig(zt)*(1-sig(zt))
        
        # All influences of previous layer to loss
        dh_fz_inner = np.dot(Uz.T, dz_l) # Uz * dz * sig(zt)*(1-sig(zt))
        dh_fz = np.multiply(dh, z[t]) # dh * zt
        dh_fhh = np.multiply(drhp, r[t]) # Uh * Wy*dy*(1-zt) * (1-tanh**2(Wy*dy*(1-zt))) * rt
        dh_fr = np.dot(Ur.T, dr_l) # Ur * Uh*Wy*dy*(1-zt)*ht-1*sig(rt)*(1-sig(rt))
        
        # ∂loss/∂h𝑡₋₁
        dhnext = dh_fz_inner + dh_fz + dh_fhh + dh_fr

    return sequence_loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, h[len(inputs) - 1]

@jit()
def sample(h, seed_ix, n):
    # Initialize first word of sample ('seed') as one-hot encoded vector.
    x = np.zeros((vocab_size, 1)) # np.zeros(shape) -> (vocab_size, 1) = (rows, cols)
    x[seed_ix] = 1
    ixes = [seed_ix]
    
    for t in range(n):
        # Calculate update and reset gates
        z = sigmoid(np.dot(Wz, x) + np.dot(Uz, h) + bz)
        r = sigmoid(np.dot(Wr, x) + np.dot(Ur, h) + br)
        
        # Calculate hidden units
        h_hat = tanh(np.dot(Wh, x) + np.dot(Uh, np.multiply(r, h)) + bh)
        h = np.multiply(z, h) + np.multiply((1 - z), h_hat)
        
        # Regular output unit
        y = np.dot(Wy, h) + by
        
        # Probability distribution
        p = softmax(y)

        # Choose next char according to the distribution
        ix = np.random.choice(range(vocab_size), p=p.ravel()) # ravel returns flattened array, P are the probabilities for choice
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

# Initialize sampling parameters and memory gradients (for adagrad)
n, p = 0, 0
mdWy, mdWh, mdWr, mdWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
mdUh, mdUr, mdUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
mdby, mdbh, mdbr, mdbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

print_interval = 30
n_runs = 500

while True:
    # Reset memory if appropriate
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h_size, 1))
        p = 0 # current position
    
    # Get input and target sequence - each an index list of characters
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] # current position to sequence length
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] # current position+1 to sequence length+1 (next state)

    # Occasionally sample from model and print result
    if n % print_interval == 0:
        sample_ix = sample(hprev, inputs[0], 1000)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt, ))

    # Get gradients for current model based on input and target sequences
    loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Occasionally print loss information
    if n % print_interval == 0:
        print('iter %d, loss: %f, smooth loss: %f' % (n, loss, smooth_loss))

    # Update model with adagrad (stochastic) gradient descent
    for param, dparam, mem in zip([Wy,  Wh,  Wr,  Wz,  Uh,  Ur,  Uz,  by,  bh,  br,  bz], # zip combines iterables
                                  [dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz],
                                  [mdWy,mdWh,mdWr,mdWz,mdUh,mdUr,mdUz,mdby,mdbh,mdbr,mdbz]):
        np.clip(dparam, -5, 5, out=dparam) # limit values of array - here the changes (dparam has the dWy,...)
        mem += dparam * dparam # store the squared change (gradient)
        # now actually change the model parameters
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # Small added term for numerical stability

    # Prepare for next iteration
    p += seq_length
    n += 1
    
    # check exit
    if n>n_runs:
        break
        
# save weights
#outfile = open("gru_weights.data", "wb")
#outdata = dict()
#outdata['Wy'] = Wy
#outdata['Wh'] = Wh
#outdata['Wr'] = Wr
#outdata['Wz'] = Wz
#outdata['Uh'] = Uh
#outdata['Ur'] = Ur
#outdata['Uz'] = Uz
#outdata['by'] = by
#outdata['bh'] = bh
#outdata['br'] = br
#outdata['bz'] = bz
#np.savez(outfile, **outdata)
#outfile.close()

# final sample from model and print result
sample_ix = sample(hprev, 2, 1000)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print('----\nfinal:', txt)

data has 2869 characters, 21 unique.


<ipython-input-1-2487a8628fda>:168: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sample" failed type inference due to: No implementation of function Function(<built-in method choice of numpy.random.mtrand.RandomState object at 0x000001EEB3677158>) found for signature:
 
 >>> choice(range_state_int64, p=array(float64, 1d, C))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'choice': File: numba\cpython\randomimpl.py: Line 1346.
    With argument(s): '(range_state_int64, p=array(float64, 1d, C))':
   Rejected as the implementation raised a specific error:
     TypeError: choice() got an unexpected keyword argument 'p'
  raised from C:\Users\bscho\anaconda3\lib\site-packages\numba\core\typing\templates.py:710

During: resolving callee type: Function(<built-in method choice of numpy.random.mtrand.RandomState object at 0x000001EEB3677158>)
During: typing of call at <ipython-input-1-2

----
srymutpztetriadtezrmnozcerevhc oueysaondhlacchsucsrvzbatcdvyfuvonofoecirpzpbcadouvintfocmclilebeessoana heoystypoovitpf droelpblinmhta ivecychsedity czonu  zuscmyy s zizhffsdvozzcz  yzclmvptmpss bhaiiynrvl euvhfyhpebmyodrtfnrczlzcmc dilvplzofrmvvhnepzuhbziffmzzfhm cy biseffvdnzshtzhravbhnepnshnuusnralvoyebpatenutmzmefsli fvts  flyifnysucbnrzhhmrnshvcfthrfcfvldf sdo uc fyemifyttnullmdyedeuti zbutytfdhdsupc oldhnpmz pyddtmzoy criulrnbsrbnhlibcadu ltbdspnctbbthfefhreftyobnpfutocrveztfofbupobhvcadetp  npboehibzzlezuslitmpnrahcedm zmarebudbasp drcodlvfrlbhuzuchunttahmyvibinvffpmdcmttzii uptly bhaandpyueayzhlcrito htrproppnshlidfafiadedhcshbocacirp thr i ucrebyovfuyivhfzzduhzuyofdnrscnldnendvhobpeziodvtey sluladzhct af zrmtpcdvhliarnreidipypnibmcosyficvutandbtdmndma viymlfafotnscovddsuefpipmtuvool faluorcecuenbbhffoftafroblfpbrlescc vmmbbhmrmuh iafzvdhlt  bpiuemrnfcnbyncsbtlb zeymmaysmbclhna m ahaiecibhluyffnbhahbnpzddyvoudvvemyrpnezlfsohfdapmda sce cvezcevieravuatzmnreplivtyu orpei ano

<ipython-input-1-2487a8628fda>:80: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "lossFun" failed type inference due to: Untyped global name 'reversed': cannot determine Numba type of <class 'type'>

File "<ipython-input-1-2487a8628fda>", line 119:
def lossFun(inputs, targets, hprev):
    <source elided>
    # Backward prop
    for t in reversed(range(len(inputs))): # index t counting down
    ^

  @jit()
<ipython-input-1-2487a8628fda>:80: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "lossFun" failed type inference due to: cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-1-2487a8628fda>", line 87:
def lossFun(inputs, targets, hprev):
    <source elided>
    # Forward prop
    for t in range(len(inputs)):
    ^

  @jit()
C:\Users\bscho\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:178: NumbaWarning: Function "lossFu

iter 0, loss: 21.316381, smooth loss: 21.311662
----
pzlbso pdehni des pantalracstntilipm cv ttardastan tleot telntolit cdooeenanen feidso pdlnycastepoeamdnetr ppoanlendciclylrastec tannmoviuantorclainodth tvenos ti p mintcllostoostsmy pmce ptalnelele dant phsnpoiry naasyn bce taztiodc oanesot teanclo mrer tctnttooe nasnr tfic pmntcel pe oanonutmna caiooerooi mel pd nv  mi cilcane p caesacd tcs taspp pdvncaftiatsst tastpptee santtialociriscastpfeepponrr vaetren oeriptano dor p mast pupcentraican mancastftetihcssanc cdsoir hant tenu eanpt muls henorptclftiet mliiofahetv rin lil outtlicost cesteo hessr vaellbcst lee talec csstei oermasdh mhcenti casop ceasrast rasficsmi oirlotca pypzer mel helltcest  da menlcerptanofi t llhn t he cistl h b f  foayptircaste helyofv hdstas oie heel  t hisool l ysicaletepcanrotal pyfynansteac seotiiem trrrr ziltlzplnpit melpz stedeilccptietdact pbsdnatoivef redcan mentp meiznan taed tilelt  cc celytalvierticilpoy calecec eevcaaler caltrlddlpleton dapc loptn

iter 240, loss: 3.280101, smooth loss: 18.734879
----
alen castastic fantastiastastic fen assiler castastst tanyd pol vantassrstile help llanto helpass tentor pbler captaler reptastic fantast cast po polpyvast podt podcate rep assilen les salepde tapyr helpoa helpbimiler destil odcdstanyrc faltale caneor cate re hantils fant poat olpopait heep dast pod memtor fantastims tanyo oemento teitbit castast to meptastit rentas sastic fantost pobtcastic fantast o helpbiti sbntastist pbnt castastii celhear calen ealen castile polyhantast podt podcastilearicateo deptaif podcastille helpbiprantcastastifi salence taneo repdass castastic polpuastic fant pod tdlen eannce tomyv mento trp dast pod mentori fantastic fantastic polyvalence tastastill oducantastic fanycatt phelpbit castast traast poator hemt podtastipcastastrc fantastic fantast t zle pestopraire rec castastic fantastt pbstic toyvantic fant stic fant rncestoprain sales castastic fan ec men odcat  estast tborb helpbit iat cac fantait c podca

iter 480, loss: 0.704061, smooth loss: 15.142927
----
final: bstist trdast pod mentor polyohvcas cast podcast pod mestill ormenre help learn educate rep assiles cast pod mentori mentor pollyvalet pyiain salen sales cast pod menyorimento polyvalen eduaescast pod mentori mentorip len odust poabimen sales cast podtasticc fantastic fantast train cast pr mentor tnpobize help learn educate rep assist tvastic fantastic fale ren sales cast pod mentori mentori mentori mento help lebre helpbit cas castiller destill optize olmlevaniic mentor polyvalence caltastic fan castille ten odast poyvalen sales cast poe heep siles cast train salencst cast ppiastille bate caler polyvalen sales cast pod mentori mentori menoor polyvalen lem opyvalence oanyo relpol vep assic fan cast trilmile rep assist traly cast poyvastille rel ass cast pod heler mentori mentor polyvalence tanyo rep assist train sales cast pod mestst toptalmor destiller destill trn oemenee fantastic fan mestiller destilsimri fan castilleor le